In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.1-8b-instant"
)

In [3]:
from langchain_core.tools import tool
import requests


# new concept
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currancy: str, target_currancy: str)-> float:

    """
    the function fetch the currancy in "Exchange Api Rate" conversion factor between a given base currancy and target currancy
    """

    API_KEY = os.getenv("CURRANCY_API_KEY")
    url =   f"https://v6.exchangerate-api.com/v6/{API_KEY}/pair/{base_currancy}/{target_currancy}"

    response = requests.get(url=url)

    return response.json()

@tool
def convert(base_currancy_value: int, conversion_rate:Annotated[float, InjectedToolArg])-> float:
    """ 
    convert base currancy and target currancy
    """

    return base_currancy_value * conversion_rate



In [4]:
get_conversion_factor.invoke({'base_currancy':'USD','target_currancy':'INR'})


{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1762128002,
 'time_last_update_utc': 'Mon, 03 Nov 2025 00:00:02 +0000',
 'time_next_update_unix': 1762214402,
 'time_next_update_utc': 'Tue, 04 Nov 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.8264}

In [6]:
convert({'base_currancy_value': 10, 'conversion_rate': 88.6768})


C:\Users\ninaw\AppData\Local\Temp\ipykernel_21532\3823946781.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  convert({'base_currancy_value': 10, 'conversion_rate': 88.6768})


886.768

In [7]:
llm_with_tool_bind = llm.bind_tools([get_conversion_factor,convert])
# llm_with_tool_bind.kwargs
# llm_with_tool_bind.kwargs['tools']


In [8]:
# i have given two question 
from langchain_core.messages import HumanMessage
messages = [HumanMessage("what is the converstion factor bet'n INR and USD , and based on that can you convert 19 USD to USD")]

In [9]:
ai_messages =llm_with_tool_bind.invoke(messages)
ai_messages.tool_calls
# THE conversion rate are wrong in convert before InjectedToolArg

[{'name': 'get_conversion_factor',
  'args': {'base_currancy': 'INR', 'target_currancy': 'USD'},
  'id': 'rh3zg6mbw',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currancy': 'USD',
   'base_currancy_value': 19,
   'target_currancy': 'USD'},
  'id': 'xvnzcn70a',
  'type': 'tool_call'}]

In [10]:
messages.append(ai_messages)

In [14]:
messages

[HumanMessage(content="what is the converstion factor bet'n INR and USD , and based on that can you convert 19 USD to USD", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rh3zg6mbw', 'function': {'arguments': '{"base_currancy":"INR","target_currancy":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'xvnzcn70a', 'function': {'arguments': '{"base_currancy":"USD","base_currancy_value":19,"target_currancy":"USD"}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 351, 'total_tokens': 409, 'completion_time': 0.06817902, 'prompt_time': 0.019346522, 'queue_time': 0.054745048, 'total_time': 0.087525542}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7b88b23a-43e0-4f10-a2c5-6818ee891899-0', tool_calls=[{'name': 'get_conversi

In [11]:
# new concept 
# InjectedToolArg = llm does not fill the aurgument as developer fill the argu

import json

for tool_call in ai_messages.tool_calls:
    # print(tool_call)
    # exicute the first tool 
    if tool_call['name'] == 'get_conversion_factor':
        tool_message_1 = get_conversion_factor.invoke(tool_call)
        # print(tool_message_1)
        # fetch the convesion rate
        converstion_rate = (json.loads(tool_message_1.content)['conversion_rate'])
        # append the tool message in messages list
        messages.append(tool_message_1)
    if tool_call['name'] == 'convert':
        # fetch the current argu
        tool_call['args']['conversion_rate'] = converstion_rate
        tool_message_2 = convert.invoke(tool_call)
        messages.append(tool_message_2)

In [12]:
messages

[HumanMessage(content="what is the converstion factor bet'n INR and USD , and based on that can you convert 19 USD to USD", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rh3zg6mbw', 'function': {'arguments': '{"base_currancy":"INR","target_currancy":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'xvnzcn70a', 'function': {'arguments': '{"base_currancy":"USD","base_currancy_value":19,"target_currancy":"USD"}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 351, 'total_tokens': 409, 'completion_time': 0.06817902, 'prompt_time': 0.019346522, 'queue_time': 0.054745048, 'total_time': 0.087525542}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7b88b23a-43e0-4f10-a2c5-6818ee891899-0', tool_calls=[{'name': 'get_conversi

In [13]:
llm_with_tool_bind.invoke(messages).content

'This conversion factor is based on the current exchange rate at the time of the API call. The conversion factor is 1 INR = 0.01127 USD. \n\nTo convert 19 USD to USD, we multiply 19 by the conversion factor. The result is approximately 0.21413 USD.'